# Introduction

In [12]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
print("done")

done


# Data Preprocessing

In [13]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print("done")

done


# Model Training

### fill_missing_values_with_ml  is a function that uses random forest with 50 trees to predict all the missing values. Since the test-set included cells with missing data, I needed to find another way of making predictions on data with missing cells. I used ML to predict missing. cells for example if fuel type is missing, we can guess from other cars that have the same engine and transmission


In [14]:

def fill_missing_values_with_ml(df):
    df = df.replace('–', np.nan)
    columns_to_predict = ['engine', 'fuel_type', 'transmission']
    features = ['brand', 'model', 'model_year']

    for column in columns_to_predict:
        train_df = df.dropna(subset=[column])
        test_df = df[df[column].isna()]

        if not test_df.empty:
            X_train = pd.get_dummies(train_df[features])
            y_train = train_df[column]
            X_test = pd.get_dummies(test_df[features])

            #DataFrame with all columns from X_train filled with zeros
            missing_cols = pd.DataFrame(0, index=X_test.index, columns=X_train.columns)

            # Update the values with X_test where columns match
            missing_cols.update(X_test)

            # Now X_test has all necessary columns
            X_test = missing_cols
            #Use Random Forest with 50 trees to fill missing values. I tried 100 but did not have enough computing power
            model = RandomForestClassifier(n_estimators=50, random_state=42)
            model.fit(X_train, y_train)

            predictions = model.predict(X_test)
            df.loc[df[column].isna(), column] = predictions

    return df

print("done")

done


In [15]:

new_train_data = fill_missing_values_with_ml(train_data)
print("done")

done


In [21]:


# ColumnTransformer for both train and test data
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [0,1,4,5,6,7])], remainder='passthrough')

# Transforming training data
X = ct.fit_transform(new_train_data.iloc[:, :-1])
y = new_train_data.iloc[:,-1].values
e
# Training the model
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
regressor.fit(X, y)

# Ensure consistent column order in test data
column_order = ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'accident']
test_data = test_data[column_order]

# Transform test data using the same ColumnTransformer
test_data_transformed = ct.transform(test_data)

# Make predictions
prediction = regressor.predict(test_data_transformed)
print("done")

MemoryError: Unable to allocate 23.7 GiB for an array with shape (54273, 58649) and data type float64

# Evaluation

# Conclusion